In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
import time
from gapless import World
%matplotlib inline

In [1]:
import sys
print (sys.version)
print (sys.executable)

2.7.12 (default, Nov 12 2018, 14:36:49) 
[GCC 5.4.0 20160609]
/usr/bin/python


In [ ]:
w = World(1)

In [ ]:
# physical parameters
q = -1.60217662e-19 # coulombs
m = 9.10938356e-31 #kg (electron)
#m = 6.6359437706294e-26 #(calcium)
kB = 1.38064852e-23 # J/K
f = 1.5e9 # Electrode frequency, in Hertz